In [2]:
!pip install -U scikit-learn xgboost albumentations bayesian-optimization

     |████████████████████████████████| 173.5 MB 11 kB/s 
     |████████████████████████████████| 102 kB 75.5 MB/s 
     |████████████████████████████████| 47.6 MB 1.8 MB/s 
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=a5d433ac6df35d8180c35e094da523733dfc9ce091574866dfddd71f14fcee77
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [6]:
import pandas as pd
import numpy as np
import os
import json
import pickle

import matplotlib.pyplot as plt

from sklearn.model_selection import ParameterSampler, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import sys
sys.path.append('./drive/MyDrive/spotify/notebooks/')
from utils import *

ROOT_PATH = "./drive/MyDrive/spotify/training/loud/10000/7/genres/"

class_order = [ 5, 20,  6, 30, 25, 26,  0,  1,  2, 27, 21, 15, 16, 17, 3, 22, 31, 7, 10, 11, 12, 13,  8, 23, 32, 28, 14, 29,  4, 18, 33,  9, 24, 34, 19]

In [4]:
rng = np.random.RandomState(42)

In [7]:
with open(ROOT_PATH + "X_train_all.pkl", 'rb') as f:
    X_train_all = pickle.load(f)

with open(ROOT_PATH + "y_train_all.pkl", 'rb') as f:
    y_train_all_u = pickle.load(f)

y_train_all = np.empty(y_train_all_u.shape)
for i, j in enumerate(class_order):
    y_train_all[:, j] = y_train_all_u[:, i]

Xy_train_all = np.hstack([X_train_all, y_train_all])
rng.shuffle(Xy_train_all)
X_train_all = Xy_train_all[:, :-35]
y_train_all = Xy_train_all[:, -35:]

In [8]:
with open(ROOT_PATH + "X_test_all.pkl", 'rb') as f:
    X_test_all = pickle.load(f)

with open(ROOT_PATH + "y_test_all.pkl", 'rb') as f:
    y_test_all_u = pickle.load(f)

y_test_all = np.empty(y_test_all_u.shape)
for i, j in enumerate(class_order):
    y_test_all[:, j] = y_test_all_u[:, i]

Xy_test_all = np.hstack([X_test_all, y_test_all])
rng.shuffle(Xy_test_all)
X_test_all = Xy_test_all[:, :-35]
y_test_all = Xy_test_all[:, -35:]

In [16]:
dtrains = [None for _ in range(7)]

for k in range(7):
    idx_train_pos = y_train_all[:, k*5:(k+1)*5] == 1
    idx_train_pos = idx_train_pos.any(axis=1)
    
    idx_train_neg = y_train_all[:, k*5:(k+1)*5] == 0
    idx_train_neg = idx_train_neg.all(axis=1)

    y_train = np.empty(X_train_all.shape[0])
    y_train[idx_train_pos] = 1
    y_train[idx_train_neg] = 0
    
    dtrains[k] = xgb.DMatrix(data=X_train_all, label=y_train)

In [17]:
dtests = [None for _ in range(7)]

for k in range(7):
    idx_test_pos = y_test_all[:, k*5:(k+1)*5] == 1
    idx_test_pos = idx_test_pos.any(axis=1)
    
    idx_test_neg = y_test_all[:, k*5:(k+1)*5] == 0
    idx_test_neg = idx_test_neg.all(axis=1)

    y_test = np.empty(X_test_all.shape[0])
    y_test[idx_test_pos] = 1
    y_test[idx_test_neg] = 0
    
    dtests[k] = xgb.DMatrix(data=X_test_all, label=y_test)

In [18]:
evals = [[(dtrain, 'train'), (dtest, 'test')] for dtrain, dtest in zip(dtrains, dtests)]

In [19]:
fixed_params = dict(
    tree_method = 'gpu_hist',
    objective = 'binary:logistic',
    eval_metric = 'logloss'
)
param_cols = ['colsample_bytree', 'gamma', 'learning_rate', 'max_depth', 'min_child_weight', 'subsample']

In [24]:
rng = np.random.RandomState(42)
for i in range(5):
    rng.randint(1e6)

In [25]:
param_grid = dict(
    learning_rate = (0.05, 0.2),
    max_depth = (2, 12),
    min_child_weight = (1, 8),
    colsample_bytree = (0.4, 1.0),
    gamma = (0, 10),
    subsample = (0.4, 1),
)

for k in range(5, 6):
    def xgb_cv_wrapper(colsample_bytree, gamma, learning_rate, max_depth, min_child_weight, subsample):
        params = fixed_params.copy()
        params['colsample_bytree'] = colsample_bytree
        params['gamma'] = gamma
        params['learning_rate'] = learning_rate
        params['max_depth'] = int(max_depth)
        params['min_child_weight'] = min_child_weight
        params['subsample'] = subsample

        res = xgboost_cv_single(
            train = dtrains[k],
            params = params,
            num_rounds = 150,
            metric_name = 'logloss',
        )
        return -res

    optimizer = BayesianOptimization(
        f = xgb_cv_wrapper,
        pbounds = param_grid,
        random_state = rng.randint(1e6),
        verbose = 2,
    )

    logger = JSONLogger(path=f"./drive/MyDrive/spotify/results_bayes_simple_test/supergenres/logs_{k}.json")
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

    optimizer.maximize(
        n_iter = 50,
        init_points = 5
    )

In [51]:
n = 3
for k in range(6, 7):
    res = []
    with open(f"./drive/MyDrive/spotify/results_bayes_simple_test/supergenres/logs_{k}.json", 'r') as f:
        for row in f:
            res.append(json.loads(row))

    res = pd.DataFrame(res)
    res = res.sort_values('target', ascending=False)
    top_params =  res.head(n)['params'].to_list()

    evals_result = [{} for _ in range(n)]
    best_model, best_score = None, np.inf
    train_steps, test_steps = None, None

    for i, temp_params in enumerate(top_params):
        params = {**fixed_params, **temp_params}
        params['max_depth'] = int(params['max_depth'])
        params['learning_rate'] = params['learning_rate']/1.5
        bst = xgb.train(
            params,
            dtrains[k],
            num_boost_round = 10000,
            early_stopping_rounds = 5,
            evals = evals[k],
            evals_result = evals_result[i],
            verbose_eval = False
        )
        if evals_result[i]['test']['logloss'][-1] < best_score:
            best_score = evals_result[i]['test']['logloss'][-1]
            best_model = bst
            train_steps = evals_result[i]['train']['logloss']
            test_steps = evals_result[i]['test']['logloss']
    
    x = range(len(train_steps))
    fig, ax = plt.subplots(figsize=(10,8))
    ax.plot(x, train_steps, label='train')
    ax.plot(x, test_steps, label='test')
    fig.legend()
    fig.savefig(f"./drive/MyDrive/spotify/results_bayes_simple_test/supergenres/curves/{k}.png", bbox_inches='tight', dpi=150)
    plt.close()

    best_model.save_model(f"./drive/MyDrive/spotify/results_bayes_simple_test/supergenres/models/model_{k}.json")
